#### M4T1: How many stores does the business have and in which countries?


In [ ]:
SELECT country_code AS country, COUNT("index") AS total_no_stores
FROM dim_store_details
WHERE store_type <> 'Web Portal'
GROUP BY country_code
ORDER BY total_no_stores DESC

#### M4T2: Which locations currently have the most stores

In [ ]:
SELECT locality, COUNT("index") AS total_no_stores
FROM dim_store_details
GROUP BY locality
ORDER BY total_no_stores DESC
LIMIT 7

#### M4T3: Which months produced the largest amount of sales

In [ ]:
SELECT
    SUM(o.product_quantity * p.product_price) AS total_sales,
    d.month
FROM
    orders_table o
JOIN
    dim_date_times d ON o.date_uuid = d.date_uuid
JOIN
    dim_products p ON o.product_code = p.product_code
GROUP BY
    d.month
ORDER BY
    total_sales DESC;

#### M4T4: How many sales are coming from online?

In [ ]:
SELECT
    COUNT(o.product_code) AS numbers_of_sales,
    SUM(o.product_quantity) AS product_quantity_count,
    CASE
        WHEN s.store_type = 'Web Portal' THEN 'Web'
        ELSE 'Offline'
    END AS location
FROM
    orders_table o
JOIN
    dim_store_details s ON o.store_code = s.store_code
GROUP BY
    location
ORDER BY
    location DESC;


#### M4T5: What percentage of sales come through each type of store?

In [ ]:
SELECT
    store_type,
    SUM(o.product_quantity * p.product_price) AS total_sales,
    ROUND((SUM(o.product_quantity * p.product_price) / (SELECT SUM(product_quantity * product_price) FROM orders_table o2 JOIN dim_products p2 ON o2.product_code = p2.product_code)) * 100, 2) AS "sales_made(%)"
FROM
    orders_table o
JOIN
    dim_store_details s ON o.store_code = s.store_code
JOIN
    dim_products p ON o.product_code = p.product_code
GROUP BY
    store_type
ORDER BY
    "sales_made(%)" DESC;

#### M4T6: Which month in each year produced the highest cost of sales?

In [ ]:
WITH monthly_sales AS (
    SELECT
        dt.year,
        dt.month,
        SUM(o.product_quantity * p.product_price) AS total_sales
    FROM
        orders_table o
    JOIN
        dim_date_times dt ON o.date_uuid = dt.date_uuid
    JOIN
        dim_products p ON o.product_code = p.product_code
    GROUP BY
        dt.year, dt.month
),
ranked_sales AS (
    SELECT
        total_sales,
        year,
        month,
        RANK() OVER (PARTITION BY year ORDER BY total_sales DESC) AS sales_rank
    FROM
        monthly_sales
)
SELECT
    total_sales,
    year,
    month
FROM
    ranked_sales
WHERE
    sales_rank = 1
ORDER BY
    total_sales DESC;

#### M4T7: What is our staff headcount?

In [ ]:
SELECT 
    SUM(staff_numbers) AS total_staff_numbers,
    country_code
FROM 
    dim_store_details
GROUP BY 
    country_code
ORDER BY 
    total_staff_numbers DESC;


#### M4T8: Which German store type is selling the most

In [ ]:
SELECT 
    SUM(ot.product_quantity * p.product_price) AS total_sales,
    sd.store_type,
    sd.country_code
FROM 
    orders_table ot
JOIN 
    dim_products p ON ot.product_code = p.product_code
JOIN 
    dim_store_details sd ON ot.store_code = sd.store_code
WHERE 
    sd.country_code = 'DE'
GROUP BY 
    sd.store_type, sd.country_code
ORDER BY 
    total_sales;


#### M4T9: How quickly is the company making sales?

In [ ]:
WITH sale_times AS (
    SELECT 
        ot.date_uuid,
        dt.year,
        dt.month,
        dt.day,
        -- Create full timestamp by combining year, month, day, and time
        TO_TIMESTAMP(dt.year || '-' || dt.month || '-' || dt.day || ' ' || dt.timestamp, 'YYYY-MM-DD HH24:MI:SS') AS full_timestamp,
        
        -- Calculate next sale's timestamp using LEAD() without partitioning by year
        LEAD(
            TO_TIMESTAMP(dt.year || '-' || dt.month || '-' || dt.day || ' ' || dt.timestamp, 'YYYY-MM-DD HH24:MI:SS')
        ) OVER (ORDER BY TO_TIMESTAMP(dt.year || '-' || dt.month || '-' || dt.day || ' ' || dt.timestamp, 'YYYY-MM-DD HH24:MI:SS') DESC) AS next_full_timestamp
    FROM 
        orders_table ot
    JOIN 
        dim_date_times dt ON ot.date_uuid = dt.date_uuid
)
SELECT 
    year,
    TO_CHAR(
        INTERVAL '1 second' * AVG(
            -- Calculate the average time taken (in seconds) between sales
            EXTRACT(EPOCH FROM full_timestamp - next_full_timestamp)
        ),
        'HH24:MI:SS'  -- Format the result as HH:MM:SS
    ) AS actual_time_taken
FROM 
    sale_times
WHERE 
    next_full_timestamp IS NOT NULL  -- Ignore rows with no next sale timestamp
GROUP BY 
    year
ORDER BY 
    actual_time_taken DESC;  -- Order by average time taken in descending order
